In [24]:
# Used to crawl bag faq
import requests
from parsel import Selector
# Used for watson
import re
import json
from ibm_watson import AssistantV1
from ibm_watson.assistant_v1 import DialogNodeOutputGenericDialogNodeOutputResponseTypeText,DialogNodeOutputTextValuesElement,DialogNodeOutput,CreateIntent,DialogNode,Example
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

Setup Watson Assistant API

In [58]:
workspace_id='PLACE_YOUR_SKILL_ID_HERE'
authenticator = IAMAuthenticator('PLACE_YOUR_API_KEY_HERE')
assistant = AssistantV1(
    version='2020-04-01',
    authenticator = authenticator
)

assistant.set_service_url('PLACE_YOUR_WORKSPACE_URL_HERE') # remove "/message" at the end

Load BAG FAQ

In [59]:
faqEntries = {}

uuids = []
intents = []
dialog_nodes = []

uuidRegex = re.compile(r"^\/en\/((.*\/)*)", re.IGNORECASE)

def extractAnswer(relativeLink):
        link = 'https://www.faq.bag.admin.ch'+relativeLink;
        response = requests.get(link)
        selector = Selector(response.text)
        return selector.css('.field-item.even p').xpath('text()').get()

def createIntentAndDialogNode(link, question, answer):
    #uuid = uuidRegex.sub('',link).replace('-','_').replace('%E2%80%99','')
    uuid = 'faq'+str(len(intents))
    intent = CreateIntent(
        intent = uuid,
        description = question,
        examples = [Example(text = question)]
    )
    intents.append(intent)
    dialogOutput = DialogNodeOutputGenericDialogNodeOutputResponseTypeText(
        response_type = 'text',
        values = [DialogNodeOutputTextValuesElement(text = answer)]
    )
    dialogNode = DialogNode(
        dialog_node = uuid,
        description = question,
        conditions = '#'+uuid,
        output = DialogNodeOutput(generic = [dialogOutput])
    )
    dialog_nodes.append(dialogNode)
    
def createIntentAndDialogNodeSlow(link, question, answer):
    uuid = 'faq'+str(len(uuids))
    print("Adding "+uuid)
    uuids.append(uuid)
    response=assistant.create_intent(
        workspace_id=workspace_id,
        intent=uuid,
        description = question,
        examples=[
            {
                'text': question,
                'description': question
            }
        ]
    ).get_result()

    dialogOutput = DialogNodeOutputGenericDialogNodeOutputResponseTypeText(
        response_type = 'text',
        values = [DialogNodeOutputTextValuesElement(text = answer)]
    )
    response=assistant.create_dialog_node(
        workspace_id=workspace_id,
        dialog_node=uuid,
        description = question,
        title = question,
        conditions='#'+uuid,
        output = DialogNodeOutput(generic = [dialogOutput])
    ).get_result()
    
def extractEntries(currentPage):
    currentEntries = currentPage.css('span a[href]')
    for e in currentEntries:
        relativeLink = e.xpath('@href').get()
        question = e.xpath('text()').get()
        answer = extractAnswer(relativeLink)
        if answer:
            createIntentAndDialogNodeSlow(relativeLink, question, answer)
        

response = requests.get('https://www.faq.bag.admin.ch/en/categories/vaccination')
selector = Selector(response.text)

extractEntries(selector)

faqPageSelectors = selector.css('a[title][href]')
faqPageUrls = []
for p in faqPageSelectors:
    url = p.xpath('@href').get()
    if "page" in url:
        response = requests.get('https://www.faq.bag.admin.ch'+url)
        selector = Selector(response.text)
        extractEntries(selector)
        

Adding faq0
Adding faq1
Adding faq2
Adding faq3
Adding faq4
Adding faq5
Adding faq6
Adding faq7
Adding faq8
Adding faq9
Adding faq10
Adding faq11
Adding faq12
Adding faq13
Adding faq14
Adding faq15
Adding faq16
Adding faq17
Adding faq18
Adding faq19
Adding faq20
Adding faq21
Adding faq22
Adding faq23
Adding faq24
Adding faq25
Adding faq26
Adding faq27
Adding faq28
Adding faq29
Adding faq30
Adding faq31
Adding faq32
Adding faq33
Adding faq34
Adding faq35
Adding faq36
Adding faq37
Adding faq38
Adding faq39
Adding faq40
Adding faq41
Adding faq42
Adding faq43
Adding faq44
Adding faq45
Adding faq46
Adding faq47


Update Watson Workspace

In [53]:
#Currently throws errors
#response=assistant.update_workspace(
#    workspace_id=workspace_id,
#    name='Updated workspace',
#    description='Example workspace updated via API',
#    dialog_nodes = dialog_nodes,
#    intents = intents
#).get_result()

#print(json.dumps(response, indent=2))